<a href="https://colab.research.google.com/github/Prathamesh-Chavan-98/RAG-Web-Scraping-/blob/main/_RAG_Using_WEB_SCRAPPING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_community langchain_chroma
!pip install pypdf
!pip install -qU langchain-openai
!pip install pymupdf
!pip install --upgrade langchain
!pip install sentence-transformers
!pip install langchainhub
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 511.1 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.0 MB/s eta 0:00:00

In [ ]:
import getpass
import os
import fitz  # PyMuPDF
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.embeddings import SentenceTransformerEmbeddings
from getpass import getpass

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

# Use the correct argument for bs_kwargs as a dictionary
loader = WebBaseLoader(
    web_path="https://www.linkedin.com/in/prathamesh-chavan-ml-engineer/",
    bs_kwargs={'parse_only': bs4.SoupStrainer()}
)


In [ ]:
text_documents = loader.load()
text_documents
print(text_documents)

[Document(metadata={'source': 'https://www.linkedin.com/in/prathamesh-chavan-ml-engineer/', 'language': 'No language found.'}, page_content='\n\n')]


In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass("Enter LangChain API Key: ")
os.environ["TOGETHER_API_KEY"] = getpass("Enter Together API Key: ")

# Load the LLM with Together API key
llm = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
)

docs = text_documents

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
splits = text_splitter.split_documents(docs)

# Create embeddings using SentenceTransformer
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


# Create a vector store (Chroma) from the document chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Create a retriever
retriever = vectorstore.as_retriever()

# Load a pre-built RAG prompt
prompt = hub.pull("rlm/rag-prompt")

# Helper function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Function to generate a response using RAG
def generate_rag_response(question):
    # Retrieve relevant documents
    relevant_docs = retriever.get_relevant_documents(question)

    if not relevant_docs:
        return "No relevant documents found."

    # Format the retrieved documents as context
    context = format_docs(relevant_docs)

    # Fill the prompt with retrieved context and the question
    formatted_prompt = prompt.format(context=context, question=question)

    # Get the response from the LLM
    response = llm(formatted_prompt)

    return response

Enter LangChain API Key: ··········
Enter Together API Key: ··········


<ipython-input-5-065e6d34e08a>:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def qa_interface():
    while True:
        # Prompt user to enter a question
        question = input("Enter your question (or type 'exit' to quit): ")

        if question.lower() == 'exit':
            print("Exiting the QA interface.")
            break

        # Generate response using the RAG pipeline
        response = generate_rag_response(question)

        # Display the answer
        print("\nAnswer:", response, "\n")

# Run the QA interface
qa_interface()


<ipython-input-5-065e6d34e08a>:38: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(question)
<ipython-input-5-065e6d34e08a>:50: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(formatted_prompt)



Answer: content=' The provided context does not include a specific title of document.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 191, 'total_tokens': 204, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None} id='run-58db0755-f310-40bd-b78c-587e4d71aab1-0' usage_metadata={'input_tokens': 191, 'output_tokens': 13, 'total_tokens': 204, 'input_token_details': {}, 'output_token_details': {}} 


Answer: content=' The process of transitioning from a prototype to production is tremendous than building the initial prototype. The first prototype, as discussed, is a basic tool to provide legitimate results. When moving from prototype to production, implementing evaluation frameworks can enhance the accuracy of the SQL Agent.' additional_kwargs={'refusal': None} response_metadata={'token_usa